In [2]:
"""Tools to help preprocessing 3D data for use in DEM
    obj -> nparray of shape: (1000,1000,3,3)
    # Definitions
        close_pooling layer: Matrix from which the distace between faces are calculated. 
                Faces that are near the target face will be included in the pooling 
        
    # Functions
        obj_to_ndarray(obj_file): Converts .obj to ndarry of faces
        
        facearray_pooling(facearray): Makes pooling layer based on face center for use with close_pool()
        
        close_pool(facearray_pooling, target face index, max_distance): Finds all faces with center 
                                                                        close to center of target face
        
        small_dataset(): Makes two small lists(data and labels) with 10 categories
            # Current only opens 10 models as well
        
        full_dataset(): All 55 categories with all models
"""

'Tools to help preprocessing 3D data for use in DEM\n    obj -> nparray of shape: (1000,1000,3,3)\n    # Definitions\n        close_pooling layer: Matrix from which the distace between faces are calculated. \n                Faces that are near the target face will be included in the pooling \n        \n    # Functions\n        obj_to_ndarray(obj_file): Converts .obj to ndarry of faces\n        \n        facearray_pooling(facearray): Makes pooling layer based on face center for use with close_pool()\n        \n        close_pool(facearray_pooling, target face index, max_distance): Finds all faces with center \n                                                                        close to center of target face\n'

In [1]:
import os
import numpy as np
from keras import backend as tf
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def facearray_pooling(facearray):
    """ Takes ndarray of faces and makes pooling layer
    # Arguments
        facearray: ndarray of faces
    
    # Returns
        (Number of faces, 3)
    """
    face_center = []
    for i in facearray:
        x = (i[0][0] + i[1][0] + i[2][0])/3
        y = (i[0][1] + i[1][1] + i[2][1])/3
        z = (i[0][2] + i[1][2] + i[2][2])/3
        face_center.append([x,y,z])
    return np.asarray(face_center)

In [3]:
def close_pool(pool, targetface, max_distance):
    """ Finds faces close to target
    # Example Use
        out = close_pool(pool, 0, .15)
        print(out.shape) -> (1144, 3)
    # Arguments
        pool: face_center pool
        
        targetface: index of face for center of search sphere
        
        max_distance: radius of search sphere
    # Returns
        ndarray of faces
    """
    target = pool[targetface]
    faces_near = []
    for i in range(0,len(pool)):
        dist = np.linalg.norm(target-pool[i])
        if dist < max_distance:
            faces_near.append(i)
    return faces_near

In [4]:
def obj_to_ndarray(obj):
    """Turns .obj format into ndarray of faces with associated points
    # Arguments
        obj: .obj text file object
    
    # Returns
        ndarray with shape of tuple: (Number of faces, 3, 3)
    """
    vertexes = []
    faces = []
    vertexes_face = []
    sep = '/'
    
    for line in obj:
        if 'v ' == line[0:2]:
            vertexes.append(line.replace('\n',''))
        elif 'f ' == line[0:2]:
            faces.append(line.replace('\n',''))
    for i in range(0,len(faces)):
        face = faces[i]
        edges = face.lstrip('f ').split(' ')
        vertex_face = []
        for edge in edges:
            vertex_index = int(edge.split(sep,1)[0])
            vertex_points = []
            for point in vertexes[vertex_index-1].lstrip('v ').split(' '):
                vertex_points.append(float(point)) 
            vertex_face.append(vertex_points)
        vertexes_face.append(vertex_face)
    return np.asarray(vertexes_face)

#file = open('ShapeNetCore.v2/02691156/371a609f050b4ed3f6497dc58a9a6f8a/models/model_normalized.obj','r')
#final = obj_to_ndarray(file)

In [5]:
def preprocess(input_obj_file):
    """Normalizes data
    # Arguments
        input_data: Array of face points
        
    # Returns
        ndarray of normalized input_data 
    """
    all_faces_in_pool = []
    all_faces = obj_to_ndarray(input_obj_file)
    pool = facearray_pooling(all_faces)
    close_indexes = close_pool(pool, 0, 1)
    for i in close_indexes:
        all_faces_in_pool.append(all_faces[i])
    faces_asarry = np.asarray(all_faces_in_pool)
    final = np.resize(faces_asarry, (1000,1000,3,3))
    return final

In [16]:
def small_dataset():
    """Makes small dataset of 10 categories
    Usage:
        data, labels = full_dataset()
    """
    # TODO: Convert to use all data
    
    core = os.listdir('ShapeNetCore.v2')
    data = []
    labels = []
    count = 0    
    for i in range(0,len(core)):
        label_number = core[i] 
        print('Label: ' + str(i))
        for b in os.listdir('ShapeNetCore.v2/' + label_number):
            print('     Model: ' + b)
            path = 'ShapeNetCore.v2/' + label_number + '/' + b + '/models/model_normalized.obj'
            file = open(path,'r')
            single = preprocess(file)
            file.close()
            one_hot_small = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            one_hot_small[i]= 1
            data.append(single)
            labels.append(one_hot_small)
            break
        count += 1
        if count > 9:
            break
    return data, labels
data, labels = small_dataset()

Label: 0
     Model: 101d0e7dbd07d8247dfd6bf7196ba84d
Label: 1
     Model: 10155655850468db78d106ce0a280f87
Label: 2
     Model: 10839d0dc35c94fcf4fb4dee5181bee
Label: 3
     Model: 10a885f5971d9d4ce858db1dc3499392
Label: 4
     Model: 133d74f10a0401317773da01b1ba21ef
Label: 5
     Model: 7a9c46a3a33c7f5b15c4530387bfd
Label: 6
     Model: 10c15151ebe3d237240ea0cdca7b391a
Label: 7
     Model: 1042d723dfc31ce5ec56aed2da084563
Label: 8
     Model: 1025dd84537d737febed407fccfaf6d8
Label: 9
     Model: 104874322c6f7a75aba93753eed86c0a
Label: 10
     Model: 8cd9b10f611ac28e866a1445c8fba9da


In [ ]:
def full_dataset():
    """Makes complete dataset of 55 categories
    Usage:
        data, labels = full_dataset()
    """
    core = os.listdir('ShapeNetCore.v2')
    data = []
    labels = []
    count = 0    
    for i in range(0,len(core)):
        label_number = core[i] 
        print('Label: ' + str(i))
        for b in os.listdir('ShapeNetCore.v2/' + label_number):
            print('     Model: ' + b)
            path = 'ShapeNetCore.v2/' + label_number + '/' + b + '/models/model_normalized.obj'
            file = open(path,'r')
            single = preprocess(file)
            file.close()
            one_hot_full = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            one_hot_full[i]= 1
            data.append(single)
            labels.append(one_hot_full)
            break
        count += 1
        if count > 9:
            break
    return data, labels